###  Хакатон

In [90]:
import pandas as pd
from catboost import Pool, CatBoostClassifier
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split 

import nltk 
import re
import pymorphy2
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from tqdm import tqdm

import sklearn.metrics as sk_met # для оценки модели 


%matplotlib inline

In [17]:
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')

In [8]:
data_path = "../data/"
file_data = "labeled.csv"
file_data = os.path.join(data_path, file_data)

In [9]:
data = pd.read_csv(file_data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comment  14412 non-null  object 
 1   toxic    14412 non-null  float64
dtypes: float64(1), object(1)
memory usage: 225.3+ KB


In [10]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [23]:
data.toxic.value_counts()

0.0    9586
1.0    4826
Name: toxic, dtype: int64

In [39]:
stop_words = set(stopwords.words('russian'))
print(stop_words)

{'чем', 'будто', 'всех', 'всю', 'кто', 'ведь', 'можно', 'он', 'где', 'бы', 'тем', 'через', 'эту', 'же', 'никогда', 'этом', 'и', 'сам', 'со', 'за', 'по', 'ли', 'а', 'все', 'вас', 'ее', 'хорошо', 'тут', 'более', 'ему', 'вдруг', 'зачем', 'этого', 'ним', 'этот', 'уже', 'при', 'у', 'лучше', 'ты', 'были', 'после', 'уж', 'том', 'между', 'нельзя', 'к', 'вот', 'их', 'да', 'три', 'два', 'про', 'куда', 'этой', 'больше', 'какой', 'наконец', 'мы', 'чтоб', 'ей', 'нас', 'или', 'здесь', 'свою', 'так', 'иногда', 'во', 'нибудь', 'нее', 'разве', 'раз', 'над', 'тогда', 'нет', 'ничего', 'может', 'себе', 'не', 'вам', 'для', 'всегда', 'тоже', 'эти', 'был', 'всего', 'на', 'какая', 'ни', 'меня', 'было', 'хоть', 'его', 'под', 'из', 'один', 'почти', 'ну', 'если', 'совсем', 'даже', 'него', 'тот', 'чтобы', 'такой', 'потому', 'сейчас', 'мой', 'до', 'что', 'была', 'моя', 'чуть', 'им', 'впрочем', 'с', 'другой', 'ж', 'ней', 'потом', 'они', 'быть', 'но', 'есть', 'чего', 'будет', 'еще', 'перед', 'теперь', 'об', 'них', '

### Приведение к нормальной форме (нижний регистр и лемматизация)

In [40]:
def lemmatized(df_train, text_col):
    # нормализация текста: приведение к нижнему регистру, удаление различных символов
    df_train[text_col] = df_train[text_col].str.lower()
    df_train[text_col] = df_train[text_col].str.replace(',', ' ')
    df_train[text_col] = df_train[text_col].str.replace('.', ' ')
    df_train[text_col] = df_train[text_col].str.replace('-', ' ')
    df_train[text_col] = df_train[text_col].str.replace(';', ' ')
    df_train[text_col] = df_train[text_col].str.replace(':', ' ')
    df_train[text_col] = df_train[text_col].str.replace('(', ' ')
    df_train[text_col] = df_train[text_col].str.replace(')', ' ')
    df_train[text_col] = df_train[text_col].str.replace('}', ' ')
    df_train[text_col] = df_train[text_col].str.replace('{', ' ')
    df_train[text_col] = df_train[text_col].str.replace('<', ' ')
    df_train[text_col] = df_train[text_col].str.replace('>', ' ')

    df_train[text_col] = df_train[text_col].str.replace('!', ' ')
    df_train[text_col] = df_train[text_col].str.replace(r'\d+', ' ')
    df_train[text_col] = df_train[text_col].str.replace(r'[\W]+', ' ')
    
    return df_train

# приведение токенов входящих в текст к нормальной форме
def norm(text, morph):  
    text_norm = ''  
    for token in nltk.word_tokenize(text):
        # print('token = ', token)
        token_norm = morph.parse(token)[0].normal_form
        if token_norm not in stop_words:        
            text_norm = text_norm + ' ' + token_norm
        # print('text_norm', text_norm)        
    return text_norm

def norm_all_df(df_train, text_col):
    # приведение к нормальной форме всех отзывов
    morph = pymorphy2.MorphAnalyzer()
    N = df_train.shape[0]
#     N = 100
    with tqdm(total=N) as progress_bar:    
        for i in range(N):
            #print('i = ', i)
            df_train.loc[i, text_col] = norm(df_train.loc[i, text_col], morph)
            progress_bar.update()
    return df_train

In [41]:
file_lemmatized = "data_lemmatized.csv"
file_lemmatized = os.path.join(data_path, file_lemmatized)

text_col = 'comment' # имя колонки с текстом

df_with_lemm = lemmatized(data, text_col)
df_with_lemm = norm_all_df(df_with_lemm, text_col)
df_with_lemm.to_csv(file_lemmatized, sep = ";", index = False)

/tmp/ipykernel_48495/4080072856.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_train[text_col] = df_train[text_col].str.replace('.', ' ')
/tmp/ipykernel_48495/4080072856.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_train[text_col] = df_train[text_col].str.replace('(', ' ')
/tmp/ipykernel_48495/4080072856.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_train[text_col] = df_train[text_col].str.replace(')', ' ')
/tmp/ipykernel_48495/4080072856.py:11: FutureWarning: The default value of regex will c

## Разделение на трейн и тест

In [56]:
data = pd.read_csv(file_lemmatized, sep=";")
data.head()

,comment,toxic
0,верблюд дебил бл,1.0
1,хохол это отдушина затюканый россиянин мол во...,1.0
2,собака собачий смерть,1.0
3,страница обновить дебил это оскорбление доказ...,1.0
4,убедить страничный пдф скрипаль отравить росс...,1.0


In [74]:
data.rename(columns={"comment": "text"}, inplace=True)
data.head()

,text,toxic
0,верблюд дебил бл,1.0
1,хохол это отдушина затюканый россиянин мол во...,1.0
2,собака собачий смерть,1.0
3,страница обновить дебил это оскорбление доказ...,1.0
4,убедить страничный пдф скрипаль отравить росс...,1.0


In [75]:
data.shape

(14412, 2)

In [78]:
data.dropna(inplace=True)
data.shape

(14411, 2)

In [79]:
df_train, df_test = train_test_split(
    data,
    test_size=0.2, 
    stratify=data["toxic"],
    random_state=42
)


In [80]:
df_train.shape, df_test.shape

((11528, 2), (2883, 2))

In [81]:
df_train.toxic.value_counts(), df_test.toxic.value_counts()

(0.0    7668
 1.0    3860
 Name: toxic, dtype: int64,
 0.0    1918
 1.0     965
 Name: toxic, dtype: int64)

In [82]:
df_train.to_csv(os.path.join(data_path, "train.csv"), index=0)
df_test.to_csv(os.path.join(data_path, "test.csv"), index=0)

### Обучим

In [83]:
train = pd.read_csv(os.path.join(data_path, "train.csv"))
train.head()

,text,toxic
0,читать закон банкротство объявить банкрот нуж...,0.0
1,сяомь это постоянно,0.0
2,светов изначально поддержать уважаемый крякло...,1.0
3,заработок работа небольшой плюс устраиваться ...,0.0
4,ездить каждый день год тыс,0.0


In [97]:
def fit_catboost(
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    catboost_params = {},
    verbose = 100
):
    learn_pool = Pool(
        X_train, 
        y_train, 
        text_features=["text"], 
        feature_names=["text"]
    )
    test_pool = Pool(
        X_test, 
        y_test, 
        text_features=["text"],
        feature_names=["text"]
    )
    catboost_default_params = {
        'iterations': 2000,
        'learning_rate': 0.015,
        'eval_metric': 'F1',
        'task_type': 'GPU',
        'use_best_model': True
    }
    catboost_default_params.update(catboost_params)
    
    model = CatBoostClassifier(**catboost_default_params)
    model.fit(learn_pool, eval_set=test_pool, verbose=verbose)
    return model



In [98]:
X_train, X_val, y_train, y_val = train_test_split(
    train[["text"]],
    train["toxic"],
    test_size=0.3, 
    stratify=train["toxic"],
    random_state=42
)
cat_boost_model = fit_catboost(X_train, X_val, y_train, y_val)

0:	learn: 0.7623269	test: 0.7916279	best: 0.7916279 (0)	total: 32.4ms	remaining: 1m 4s
100:	learn: 0.7909955	test: 0.7831498	best: 0.7930712 (23)	total: 2.55s	remaining: 48s
200:	learn: 0.7963818	test: 0.7891538	best: 0.7930712 (23)	total: 4.92s	remaining: 44.1s
300:	learn: 0.8027340	test: 0.7987073	best: 0.7987073 (283)	total: 7.12s	remaining: 40.2s
400:	learn: 0.8048457	test: 0.7994467	best: 0.7998155 (385)	total: 9.35s	remaining: 37.3s
500:	learn: 0.8054401	test: 0.7998159	best: 0.8001842 (499)	total: 11.5s	remaining: 34.4s
600:	learn: 0.8070308	test: 0.7998159	best: 0.8001842 (499)	total: 13.8s	remaining: 32.2s
700:	learn: 0.8083097	test: 0.8001842	best: 0.8007363 (667)	total: 16.1s	remaining: 29.9s
800:	learn: 0.8096768	test: 0.8012879	best: 0.8012879 (752)	total: 18.6s	remaining: 27.8s
900:	learn: 0.8118025	test: 0.8027586	best: 0.8027586 (849)	total: 20.9s	remaining: 25.5s
1000:	learn: 0.8134116	test: 0.8016567	best: 0.8027586 (849)	total: 23.3s	remaining: 23.3s
1100:	learn: 0.8

### Проверим f1 на данных которые модель не видела

In [99]:
test = pd.read_csv(os.path.join(data_path, "test.csv"))
test.head()

,text,toxic
0,увы увы это нужно придумать сверхъестественны...,0.0
1,просто уметь читать строка,0.0
2,срочный избиение з б н бесплатный смс подписы...,1.0
3,добби свободный ох спасибо добрый внученька,0.0
4,почему собака подходить нюхать твой жопа кот ...,1.0


In [100]:
X_test, y_test = test[["text"]], test["toxic"]
X_test.head()

,text
0,увы увы это нужно придумать сверхъестественны...
1,просто уметь читать строка
2,срочный избиение з б н бесплатный смс подписы...
3,добби свободный ох спасибо добрый внученька
4,почему собака подходить нюхать твой жопа кот ...


In [101]:
y_pred = cat_boost_model.predict(X_test)
print(
    'F1-score на тестовой выборке: {:.3f} \n'
    .format(
        sk_met.f1_score(
            y_test, 
            y_pred, 
            average = 'macro')
    )
)

F1-score на тестовой выборке: 0.844 



In [102]:
cat_boost_model.save_model("hack_model")